In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
base = '../output/'

historical_prices_df = pd.read_csv(base + 'nasdaq_historical_prices.csv')
historical_prices_df = historical_prices_df.drop(columns='Unnamed: 0')
historical_prices_df

,Company,Q1 Date,Q1 Price,Q2 Date,Q2 Price,Q3 Date,Q3 Price
0,DOCU,2020-01-31,78.5100,2020-04-30,104.7500,2020-07-31,216.830000
1,ZM,2020-01-31,76.3000,2020-04-30,135.1700,2020-07-31,253.910000
2,DXCM,2019-12-31,218.7400,2020-03-31,269.2700,2020-06-30,405.400000
3,SPLK,2020-01-31,155.2600,2020-04-30,140.3600,2020-07-31,209.820000
4,SGEN,2019-12-31,114.2600,2020-03-31,115.3800,2020-06-30,169.920000
...,...,...,...,...,...,...,...
98,NVDA,2020-01-26,NaN,2020-04-26,NaN,2020-07-26,NaN
99,PAYX,2020-02-29,NaN,2020-05-31,NaN,2020-08-31,76.470001
100,KLAC,2019-12-31,175.6855,2020-03-31,142.4356,2020-06-30,193.646300
101,CSCO,2020-01-25,NaN,2020-04-25,NaN,2020-07-25,NaN


In [9]:
financial_ratios_df = pd.read_csv(base + 'nasdaq_financial_ratios.csv')
financial_ratios_df = financial_ratios_df.drop(columns='Unnamed: 0')
financial_ratios_df

,Company,Q1 P/E,Q2 P/E,Q3 P/E,Q1 ROE,Q2 ROE,Q3 ROE,Q1 Date,Q2 Date,Q3 Date
0,DOCU,-299.597287,-400.948571,-620.874031,-0.086774,-0.091441,-0.128634,2020-01-31,2020-04-30,2020-07-31
1,ZM,1191.100353,1473.689789,405.682498,0.018393,0.030029,0.155144,2020-01-31,2020-04-30,2020-07-31
2,DXCM,218.504035,1273.281759,849.326134,0.105031,0.021295,0.034198,2019-12-31,2020-03-31,2020-06-30
3,SPLK,-1064.434087,-72.359266,-127.625338,-0.011367,-0.168823,-0.156529,2020-01-31,2020-04-30,2020-07-31
4,SGEN,759.556952,-118.084957,-1390.521355,0.013767,-0.095379,-0.011681,2019-12-31,2020-03-31,2020-06-30
...,...,...,...,...,...,...,...,...,...,...
98,NVDA,156.637749,201.356865,419.415270,0.077925,0.070005,0.044703,2020-01-26,2020-04-26,2020-07-26
99,PAYX,77.441170,116.583405,130.569997,0.128451,0.079349,0.076153,2020-02-29,2020-05-31,2020-08-31
100,KLAC,73.227875,285.357774,73.578936,0.142543,0.034000,0.154292,2019-12-31,2020-03-31,2020-06-30
101,CSCO,71.068582,64.528627,74.669499,0.080995,0.077686,0.069515,2020-01-25,2020-04-25,2020-07-25


In [11]:
financial_growth_df = pd.read_csv(base + 'nasdaq_financial_growth.csv')
financial_growth_df

,Company,Q1 Op Income Growth,Q2 Op Income Growth,Q3 Op Income Growth,Q1 EPS Growth,Q2 EPS Growth,Q3 EPS Growth,Q1 Rev Growth,Q2 Rev Growth,Q3 Rev Growth,Q1 Date,Q2 Date,Q3 Date
0,DOCU,-4.235300e+07,-4.185300e+07,-5.863500e+07,-0.26205,-0.26126,-0.34923,2.748950e+08,2.970170e+08,3.422090e+08,2020-01-31,2020-04-30,2020-07-31
1,ZM,1.055300e+07,2.338500e+07,1.881040e+08,0.06406,0.09172,0.62588,1.882510e+08,3.281670e+08,6.635200e+08,2020-01-31,2020-04-30,2020-07-31
2,DXCM,1.015000e+08,3.360000e+07,6.240000e+07,1.00108,0.21148,0.47732,4.628000e+08,4.051000e+08,4.518000e+08,2019-12-31,2020-03-31,2020-06-30
3,SPLK,-7.769000e+06,-2.886120e+08,-2.394590e+08,-0.14586,-1.93977,-1.64403,7.911820e+08,4.340770e+08,4.916580e+08,2020-01-31,2020-04-30,2020-07-31
4,SGEN,-3.841400e+07,-1.123550e+08,-9.396500e+07,0.15043,-0.97709,-0.12220,2.898040e+08,2.345140e+08,2.779980e+08,2019-12-31,2020-03-31,2020-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,NVDA,9.900000e+08,9.760000e+08,6.510000e+08,1.53140,1.47428,0.99361,3.105000e+09,3.080000e+09,3.866000e+09,2020-01-26,2020-04-26,2020-07-26
99,PAYX,4.701000e+08,2.996000e+08,2.840000e+08,0.98199,0.61187,0.58566,1.142700e+09,9.151000e+08,9.322000e+08,2020-02-29,2020-05-31,2020-08-31
100,KLAC,4.628310e+08,1.534260e+08,4.531580e+08,2.39916,0.49915,2.63182,1.509453e+09,1.423964e+09,1.459593e+09,2019-12-31,2020-03-31,2020-06-30
101,CSCO,3.380000e+09,3.414000e+09,3.247000e+09,0.67559,0.65378,0.62140,1.200500e+10,1.198300e+10,1.215400e+10,2020-01-25,2020-04-25,2020-07-25
